# Build pipeline using schedule

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define and load `CommandComponent` from YAML
- Create `Pipeline` using loaded component.
- Set the schedule of `Pipeline`.

**Motivations** - This notebook explains how to create a pipeline using schedule.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.ml import MLClient, dsl, Input
from azure.ml.entities import (
    CronSchedule,
    RecurrenceSchedule,
    RecurrencePattern,
    ScheduleStatus,
    load_component,
)

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Define and create components into workspace
## 2.1 Load components definition from YAML

In [ ]:
parent_dir = "."
train_model = load_component(yaml_file=parent_dir + "/train_model.yml")
score_data = load_component(yaml_file=parent_dir + "/score_data.yml")
eval_model = load_component(yaml_file=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

In [ ]:
# Print the component as yaml
print(train_model)

# 3. Sample pipeline job
## 3.1 Build pipeline

In [ ]:
# Construct pipeline
@dsl.pipeline(
    default_compute="cpu-cluster",
    description="E2E dummy train-score-eval pipeline using schedule",
)
def pipeline_using_schedule(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, test_data=test_input
    )
    score_with_sample_data.outputs.score_output.mode = "upload"

    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "trained_model": train_with_sample_data.outputs.model_output,
        "scored_data": score_with_sample_data.outputs.score_output,
        "evaluation_report": eval_with_sample_data.outputs.eval_output,
    }

In [ ]:
pipeline = pipeline_using_schedule(
    training_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    test_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
)

## 3.2 Configure schedule of pipeline job

In [ ]:
# create a cron schedule fire at 17:00 PM and 17:30 PM with UTC timezone every day
cron_schedule = CronSchedule(
    expression="0,30 17 * * *",
    start_time="2022-04-21T01:15:00",
    time_zone="Universal Coordinated Time",
    status=ScheduleStatus.ENABLED,
)
pipeline.schedule = cron_schedule

In [ ]:
# create a recurrence schedule fire at 10:00 AM and 10:01AM with PST timezone every day
# pattern = RecurrencePattern(hours=10, minutes=[0, 1])
# recurrence_schedule = RecurrenceSchedule(
#     frequency='day',
#     interval=1,
#     start_time="2022-04-21T01:15:00",
#     time_zone="Pacific Standard Time",
#     status=ScheduleStatus.ENABLED
# )
# pipeline.schedule = recurrence_schedule

In [ ]:
print(pipeline)

## 3.2 Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="pipeline_samples"
)

Note: Job will be run as per the defined schedule. It might not run immediately.

In [ ]:
pipeline_job

# Next Steps
You can see further examples of running a pipeline job [here](../)